In [1]:
import pandas as pd
import os
import datetime
import json
from functools import reduce

In [2]:
ROOT_DIR = r"C:\dev\backend\tasks\samples"
#asof_date = datetime.date(2022, 12, 7)
#DIR_PATH = os.path.join(ROOT_DIR, asof_date.strftime("%d%m%Y"))

#DIR_PATH

In [3]:
# Read json files
sebi_detail_json_fp = os.path.join(ROOT_DIR, "sebi_detail.json")
sebi_overview_json_fp = os.path.join(ROOT_DIR, "sebi_overview.json")

lst_files = [sebi_detail_json_fp, sebi_overview_json_fp]
lst_files_content = []

for f in lst_files:
    # opening JSON file
    with open(f) as json_file:
        lst_files_content.append(json.load(json_file))

In [7]:
# Read Sebi Overview & Details
sebi_details_data = lst_files_content[0]
sebi_overview_data = lst_files_content[1]
lst_sebi_data_df = []
lst_sebi_latest_data_df = []

for so in sebi_overview_data:
    amc_name = so['name']
    amc_sebi_nr = so['sebi_nr']
    amc_detail = sebi_details_data.get(amc_sebi_nr)
    amc_growth_data = amc_detail.get('growth')
    amc_flow_data = amc_detail.get('flow')
    if amc_growth_data['date'] == amc_flow_data['date']:
        combined_data = amc_growth_data | amc_flow_data
    else:
        print(f'date data not matching for {amc_name}')
        if len(amc_growth_data['date']) == 0:
            combined_data = amc_flow_data
        # TODO: Add handling here to make sure we map the dates accordingly to data
    
    columns = list(combined_data.keys())
    amc_data_df = pd.DataFrame(combined_data, columns=columns)
    amc_data_df.insert(loc = 0, column = 'amc_sebi_nr', value = amc_sebi_nr)
    amc_data_df.insert(loc = 0, column = 'amc_name', value = amc_name)
    amc_data_df['monthly_netflow_cr'] = amc_data_df['monthly_inflow'] - amc_data_df['monthly_outflow']
    #print(amc_data_df)
    lst_sebi_data_df.append(amc_data_df)
    lst_sebi_latest_data_df.append(amc_data_df.tail(1))


print('All good!')

date data not matching for ALDER CAPITAL ADVISORS LLP
date data not matching for ICICI PRUDENTIAL ASSET MANAGEMENT COMPANY LTD
All good!


In [8]:
# Combine all the dfs into one
final_df = pd.concat(lst_sebi_data_df)
final_df.to_excel(r'C:\dev\backend\tasks\samples\sebi_report.xlsx', index=False)

# Get latest month data into one
final_latest_df = pd.concat(lst_sebi_latest_data_df)
final_latest_df.to_excel(r'C:\dev\backend\tasks\samples\sebi_report_last_available_month.xlsx', index=False)
